In [109]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

In [110]:
datasheet = "TATACONSUM_data.xlsx"
df = pd.read_excel(datasheet)

In [111]:
def data_processor(df):
    """
    Processes the input DataFrame 'df' containing stock data, 
    reads NIFTY data, calculates various metrics, 
    and returns a DataFrame 'proc_df' with expected return from 
    a rolling linear regression model.
    """
    
    # 1. Create a copy for calculations that won't clutter the original
    df = df.copy()

    #event_date = pd.to_datetime(event_date)
    
    # 2. Basic columns for the output DataFrame
    proc_df = pd.DataFrame()
    proc_df['Date'] = df['Date']
    proc_df['Stock_Price'] = df['Close']
    
    # 3. Stock daily returns
    df['Stock_Daily_Returns'] = df['Close'].pct_change()
    proc_df['Stock_Daily_Returns'] = df['Stock_Daily_Returns']
    
    # 4. Volume metrics
    proc_df['Volume'] = df['Volume']
    proc_df['Volume_change'] = df['Volume'].diff()
    window = 10
    df['Volume_mean'] = df['Volume'].rolling(window).mean()
    df['Volume_std'] = df['Volume'].rolling(window).std()
    proc_df['Volume_pct_change'] = df['Volume'].pct_change() 
    proc_df['Volume_zscore'] = (df['Volume'] - df['Volume_mean']) / df['Volume_std']
    
    # 5. Log returns & rolling volatility (std dev)
    df['Log_Returns'] = np.log(df['Close'] / df['Close'].shift(1))
    proc_df['Volatility'] = df['Log_Returns'].rolling(window).std()
    
    # 6. Read NIFTY data (assumes same date alignment)
    nifty_df = pd.read_excel("NIFTY_data.xlsx")
    nifty_df['Nifty_Daily_Returns'] = nifty_df['Close'].pct_change()
    
    # Merge or align by Date if needed. If same length + same ordering, you can do direct assignment.
    # Example: If they have the same index length and order:
    df['Nifty_Daily_Returns'] = nifty_df['Nifty_Daily_Returns']
    #    or if you must merge by Date:
    
    # 7. Drop NaN rows from the merges/changes
    df.dropna(subset=['Stock_Daily_Returns', 'Nifty_Daily_Returns'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    # 8. Prepare for rolling regression
    # We'll create a new column in proc_df for Expected_Return
    proc_df['Expected_Return'] = np.nan
    
    # Set rolling window size for the regression
    n = 10
    
    # 9. Rolling regression
    for i in range(n, len(df)):
        # Past n rows are [i-n, i-1], inclusive
        train_df = df.iloc[i-n : i]
        
        # X_train must be 2D
        X_train = train_df[['Nifty_Daily_Returns']].values
        y_train = train_df['Stock_Daily_Returns'].values
        
        # We'll predict for row i (the "today" row),
        # using today's Nifty return to forecast today's Stock return
        # If you want next-day forecast, shift i accordingly
        X_pred = df.loc[i, ['Nifty_Daily_Returns']].values.reshape(1, -1)
        
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        predicted_return = model.predict(X_pred)[0]
        
        # Put the result in proc_df for the matching date
        # We'll find the matching date in df
        row_date = df.loc[i, 'Date']
        
        # We need the index in proc_df that matches 'row_date'
        proc_index = proc_df[proc_df['Date'] == row_date].index
        if not proc_index.empty:
            proc_df.at[proc_index[0], 'Expected_Return'] = predicted_return
        prev_close = df.loc[i-1, 'Close']
        expected_price = prev_close * (1 + predicted_return)
        proc_df.at[i, 'Expected_Price'] = expected_price

    '''proc_df['Binary_Date_Flag'] = np.nan
    for i in range (len(proc_df)):
        date = proc_df.at[i, 'Date']
        if((date < event_date) and (event_date - date).days <= 30):
            proc_df.at[i, 'Binary_Date_Flag'] = 1
        else:
            proc_df.at[i, 'Binary_Date_Flag'] = 0'''
            
        
    proc_df['Market_Price'] = nifty_df['Close']   
    proc_df.dropna(inplace = True)
        
    
    return proc_df
    

In [112]:
data_processor(df).to_excel("processed_data.xlsx")

In [113]:
processed_data = data_processor(df)

In [114]:
processed_data.columns

Index(['Date', 'Stock_Price', 'Stock_Daily_Returns', 'Volume', 'Volume_change',
       'Volume_pct_change', 'Volume_zscore', 'Volatility', 'Expected_Return',
       'Expected_Price', 'Market_Price'],
      dtype='object')

In [115]:
def detect_insider_trading(proc_df, contamination=0.03, random_state=42):
    """
    Applies an Isolation Forest model to detect potential insider trading anomalies 
    in a preprocessed DataFrame `proc_df`.

    Parameters
    ----------
    proc_df : pd.DataFrame
        The DataFrame returned by data_processor(...), containing numeric columns 
        for features such as 'Stock_Daily_Returns', 'Volume_zscore', etc.
    feature_cols : list of str, optional
        The list of column names in proc_df to use as features for the Isolation Forest.
        If None, a default set of columns will be used.
    contamination : float, optional
        The proportion of outliers in the data set. Used by IsolationForest for thresholding.
        Default is 0.01 (1%).
    random_state : int, optional
        Random seed for reproducibility.

    Returns
    -------
    pd.DataFrame
        A copy of proc_df with two extra columns:
        - 'Anomaly_Label': +1 (normal) or -1 (anomalous)
        - 'Anomaly_Score': continuous score (lower = more anomalous)
    """
    
    # 1) Default feature columns if not provided
    feature_cols = [
            'Stock_Daily_Returns',
            'Volume_zscore',
            'Volatility',]
    
    # Create a copy so we don't modify the original DataFrame
    df_model = proc_df.copy()
    
    # 2) Drop rows with NaN in the feature columns
    df_model.dropna(subset=feature_cols, inplace=True)
    
    # 3) Create the feature matrix X
    X = df_model[feature_cols].values
    
    # 4) Build and train the Isolation Forest
    iso_forest = IsolationForest(
        n_estimators=100,
        contamination=contamination,
        random_state=random_state
    )
    iso_forest.fit(X)
    
    # 5) Predict anomaly labels and get anomaly scores
    anomaly_labels = iso_forest.predict(X)         # +1 or -1
    anomaly_scores = iso_forest.decision_function(X)  # higher = more normal, lower = more anomalous
    
    # 6) Add these to df_model
    df_model['Anomaly_Label'] = anomaly_labels
    df_model['Anomaly_Score'] = anomaly_scores
    
    # 7) Merge back to the original proc_df by index
    #    so we align rows that had valid feature data
    df_result = proc_df.copy()
    df_result = df_result.merge(
        df_model[['Anomaly_Label', 'Anomaly_Score']],
        left_index=True,
        right_index=True,
        how='left'
    )
    
    return df_result

In [116]:
detect_insider_trading(processed_data)

,Date,Stock_Price,Stock_Daily_Returns,Volume,Volume_change,Volume_pct_change,Volume_zscore,Volatility,Expected_Return,Expected_Price,Market_Price,Anomaly_Label,Anomaly_Score
11,2021-03-02,612.856750,0.012489,3051386,521143.0,0.205966,-0.739066,0.016092,-0.000684,610.591399,8767.250000,1,0.178690
12,2021-03-03,617.266602,0.007196,3601961,550575.0,0.180434,-0.383187,0.016139,-0.003696,618.866593,8683.849609,1,0.212821
13,2021-03-04,611.645264,-0.009107,2621353,-980608.0,-0.272243,-0.704443,0.015819,0.002592,612.312652,8844.599609,1,0.187307
14,2021-03-05,603.455750,-0.013389,2839628,218275.0,0.083268,-0.603164,0.016018,0.001091,603.958905,8956.750000,1,0.185361
15,2021-03-08,597.059265,-0.010600,1939925,-899703.0,-0.316838,-0.900343,0.016415,0.000834,597.577680,8996.250000,1,0.168854
...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,2025-02-07,1021.200012,-0.002101,941811,-230668.0,-0.196735,-0.889246,0.026734,0.003908,1028.425260,10831.400391,1,0.075645
986,2025-02-10,1027.500000,0.006169,1418998,477187.0,0.506670,-0.474225,0.026488,0.007075,1035.073340,10746.049805,1,0.094160
987,2025-02-11,1013.799988,-0.013333,1204012,-214986.0,-0.151505,-0.619072,0.027176,0.007371,1018.416021,10724.400391,1,0.078863
988,2025-02-12,1029.750000,0.015733,1082392,-121620.0,-0.101012,-0.650881,0.027369,0.004553,1036.590289,10640.950195,1,0.065206


In [117]:
detect_insider_trading(processed_data).to_excel("results_data.xlsx")

In [118]:
def detect_insider_trading_dbscan(proc_df, eps=0.9, min_samples=5):
    """
    Applies DBSCAN clustering to detect potential insider trading anomalies in a preprocessed DataFrame.
    
    Parameters
    ----------
    proc_df : pd.DataFrame
        Preprocessed DataFrame (e.g., from your data_processor function) that contains columns like
        'Stock_Daily_Returns', 'Volume_zscore', 'Volatility', etc.
    feature_cols : list of str, optional
        The list of feature column names to use. If None, defaults to:
            ['Stock_Daily_Returns', 'Volume_zscore', 'Volatility']
    eps : float, optional
        The maximum distance between two samples for one to be considered as in the neighborhood of the other.
        (Default is 0.5)
    min_samples : int, optional
        The number of samples (or total weight) in a neighborhood for a point to be considered as a core point.
        (Default is 5)
    
    Returns
    -------
    pd.DataFrame
        A copy of proc_df with two extra columns:
            - 'DBSCAN_Label': Cluster label from DBSCAN (-1 indicates an anomaly).
            - 'DBSCAN_Score': Currently set to NaN (DBSCAN doesn't provide a continuous score by default).
    """
    # 1. Set default feature columns if not provided.
    feature_cols = ['Stock_Daily_Returns', 'Volume_zscore', 'Volatility']
    
    # 2. Create a working copy and drop rows with missing values in these features.
    df_model = proc_df.copy()
    df_model.dropna(subset=feature_cols, inplace=True)
    
    # 3. Extract feature matrix
    X = df_model[feature_cols].values
    
    # 4. Scale features (DBSCAN is sensitive to scale)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # 5. Initialize and fit DBSCAN
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    dbscan_labels = dbscan.fit_predict(X_scaled)
    
    # 6. Save the DBSCAN results.
    # DBSCAN assigns -1 to points considered as noise (i.e., anomalies)
    df_model['DBSCAN_Label'] = dbscan_labels
    # Optionally, you could compute a custom score (e.g., distance to cluster centroid)
    # For now, we set 'DBSCAN_Score' to NaN.
    df_model['DBSCAN_Score'] = np.nan
    
    # 7. Merge the DBSCAN results back into the original proc_df.
    df_result = proc_df.copy()
    # We merge on index so that rows with valid feature data get the DBSCAN info;
    # rows that were dropped remain NaN in these new columns.
    df_result = df_result.merge(
        df_model[['DBSCAN_Label', 'DBSCAN_Score']],
        left_index=True,
        right_index=True,
        how='left'
    )
    
    return df_result

In [119]:
result_df = detect_insider_trading_dbscan(processed_data)

# To see which days are flagged as anomalies:
anomalies = result_df[result_df['DBSCAN_Label'] == -1]
DB_scan_results = anomalies[['Date', 'Stock_Price', 'Stock_Daily_Returns', 'Volume_zscore', 'Volatility', 'DBSCAN_Label']]
DB_scan_results.to_excel(("DBSCAN_results_data.xlsx"))

print(anomalies[['Date', 'Stock_Price', 'Stock_Daily_Returns', 'Volume_zscore', 'Volatility', 'DBSCAN_Label']])

          Date  Stock_Price  Stock_Daily_Returns  Volume_zscore  Volatility  \
29  2021-03-30   609.513184             0.000955       2.805355    0.018134   
42  2021-04-20   659.134766             0.024093       1.963469    0.022786   
51  2021-05-04   625.359131            -0.047039       1.636778    0.020009   
54  2021-05-07   609.610046            -0.036901       2.588555    0.020630   
192 2021-11-26   747.318665            -0.050879       2.189166    0.021572   
194 2021-11-30   758.917786             0.016449       2.287808    0.021699   
240 2022-02-03   719.441589            -0.005859       2.261694    0.020301   
255 2022-02-24   660.568542            -0.062980       1.457985    0.024184   
256 2022-02-25   691.564575             0.046923      -0.432937    0.028794   
259 2022-03-03   669.682129            -0.033685      -0.012055    0.031037   
260 2022-03-04   653.014404            -0.024889       1.537067    0.029959   
262 2022-03-08   666.416748             0.036145    